In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile9 = pd.read_pickle("FFF_Train_Decile9.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile9.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile9, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 15s 5ms/step - loss: 0.3029 - acc: 0.8890 - val_loss: 0.2543 - val_acc: 0.9058
Epoch 51/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2916 - acc: 0.8966 - val_loss: 0.2554 - val_acc: 0.9110
Epoch 52/75
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2994 - acc: 0.8946 - val_loss: 0.2532 - val_acc: 0.9084
Epoch 53/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2998 - acc: 0.8946 - val_loss: 0.2534 - val_acc: 0.9097
Epoch 54/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2970 - acc: 0.8884 - val_loss: 0.2521 - val_acc: 0.9084
Epoch 55/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3005 - acc: 0.8910 - val_loss: 0.2540 - val_acc: 0.9123
Epoch 56/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2983 - acc: 0.8907 - val_loss: 0.2494 - val_acc: 0.9031
Epoch 57/75
3055/3055 [==============================] - 14s

3055/3055 [==============================] - 16s 5ms/step - loss: 0.2487 - acc: 0.8982 - val_loss: 0.2127 - val_acc: 0.9084
Epoch 34/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2567 - acc: 0.8959 - val_loss: 0.2128 - val_acc: 0.9084
Epoch 35/75
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2576 - acc: 0.8926 - val_loss: 0.2131 - val_acc: 0.9084
Epoch 36/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2598 - acc: 0.8962 - val_loss: 0.2129 - val_acc: 0.9084
Epoch 37/75
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2504 - acc: 0.8962 - val_loss: 0.2128 - val_acc: 0.9084
Epoch 38/75
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2498 - acc: 0.8998 - val_loss: 0.2125 - val_acc: 0.9084
Epoch 39/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2487 - acc: 0.9002 - val_loss: 0.2125 - val_acc: 0.9084
Epoch 40/75
3055/3055 [==============================] - 15s

Epoch 18/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2975 - acc: 0.8884 - val_loss: 0.2513 - val_acc: 0.8953
Epoch 19/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3002 - acc: 0.8786 - val_loss: 0.2417 - val_acc: 0.8966
Epoch 20/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2821 - acc: 0.8894 - val_loss: 0.2388 - val_acc: 0.8992
Epoch 21/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2820 - acc: 0.8923 - val_loss: 0.2339 - val_acc: 0.9031
Epoch 22/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2729 - acc: 0.8953 - val_loss: 0.2340 - val_acc: 0.9005
Epoch 23/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2756 - acc: 0.8943 - val_loss: 0.2325 - val_acc: 0.9005
Epoch 24/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2672 - acc: 0.8979 - val_loss: 0.2555 - val_acc: 0.8992
Epoch 25/100
3055/3055 [=================

Epoch 77/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2473 - acc: 0.9051 - val_loss: 0.2160 - val_acc: 0.9058
Epoch 78/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2609 - acc: 0.8966 - val_loss: 0.2155 - val_acc: 0.9058
Epoch 79/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2531 - acc: 0.9011 - val_loss: 0.2152 - val_acc: 0.9071
Epoch 80/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2537 - acc: 0.9008 - val_loss: 0.2149 - val_acc: 0.9058
Epoch 81/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2424 - acc: 0.9057 - val_loss: 0.2153 - val_acc: 0.9071
Epoch 82/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2438 - acc: 0.9005 - val_loss: 0.2150 - val_acc: 0.9084
Epoch 83/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2454 - acc: 0.9041 - val_loss: 0.2153 - val_acc: 0.9071
Epoch 84/100
3055/3055 [=================

...

3055/3055 [==============================] - 15s 5ms/step - loss: 0.2476 - acc: 0.8985 - val_loss: 0.2135 - val_acc: 0.9084
Epoch 46/50
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2563 - acc: 0.8926 - val_loss: 0.2136 - val_acc: 0.9084
Epoch 47/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2504 - acc: 0.8933 - val_loss: 0.2138 - val_acc: 0.9084
Epoch 48/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2511 - acc: 0.9034 - val_loss: 0.2136 - val_acc: 0.9084
Epoch 49/50
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2485 - acc: 0.9002 - val_loss: 0.2134 - val_acc: 0.9084
Epoch 50/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2504 - acc: 0.8979 - val_loss: 0.2135 - val_acc: 0.9084
Test accuracy: 0.908376963975
Train on 3055 samples, validate on 764 samples
Epoch 1/125
3055/3055 [==============================] - 21s 7ms/step - loss: 0.7020 - acc: 0.4867 - val_loss: 0.6788 -

3055/3055 [==============================] - 16s 5ms/step - loss: 0.2953 - acc: 0.8959 - val_loss: 0.2649 - val_acc: 0.8992

Epoch 00061: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 62/125
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2915 - acc: 0.9002 - val_loss: 0.2642 - val_acc: 0.9005
Epoch 63/125
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2980 - acc: 0.9008 - val_loss: 0.2636 - val_acc: 0.9018
Epoch 64/125
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2920 - acc: 0.9034 - val_loss: 0.2636 - val_acc: 0.9018
Epoch 65/125
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2957 - acc: 0.8969 - val_loss: 0.2634 - val_acc: 0.9018
Epoch 66/125
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2892 - acc: 0.8992 - val_loss: 0.2632 - val_acc: 0.9018
Epoch 67/125
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2928 - acc: 0.8985 - val_loss: 0.2634 - val_a

Epoch 5/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.3032 - acc: 0.8956 - val_loss: 0.2635 - val_acc: 0.9058
Epoch 6/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.3019 - acc: 0.8907 - val_loss: 0.2511 - val_acc: 0.9058
Epoch 7/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2872 - acc: 0.8923 - val_loss: 0.2400 - val_acc: 0.9058
Epoch 8/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2896 - acc: 0.8920 - val_loss: 0.2349 - val_acc: 0.9005
Epoch 9/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2796 - acc: 0.8946 - val_loss: 0.2249 - val_acc: 0.9058
Epoch 10/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2790 - acc: 0.8956 - val_loss: 0.2260 - val_acc: 0.9005
Epoch 11/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2628 - acc: 0.8966 - val_loss: 0.2631 - val_acc: 0.8887
Epoch 12/100
3055/3055 [======================

3055/3055 [==============================] - 11s 3ms/step - loss: 0.2400 - acc: 0.9074 - val_loss: 0.2109 - val_acc: 0.9136
Epoch 85/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2366 - acc: 0.9002 - val_loss: 0.2110 - val_acc: 0.9123
Epoch 86/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2344 - acc: 0.9087 - val_loss: 0.2107 - val_acc: 0.9123
Epoch 87/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2337 - acc: 0.9061 - val_loss: 0.2107 - val_acc: 0.9123
Epoch 88/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2359 - acc: 0.9047 - val_loss: 0.2107 - val_acc: 0.9110
Epoch 89/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2395 - acc: 0.9018 - val_loss: 0.2106 - val_acc: 0.9110
Epoch 90/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2318 - acc: 0.9070 - val_loss: 0.2112 - val_acc: 0.9123
Epoch 91/100
3055/3055 [==============================] 

Epoch 44/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2522 - acc: 0.8989 - val_loss: 0.2106 - val_acc: 0.9058
Epoch 45/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2554 - acc: 0.8969 - val_loss: 0.2116 - val_acc: 0.9071
Epoch 46/50
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2562 - acc: 0.8966 - val_loss: 0.2095 - val_acc: 0.9058
Epoch 47/50
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2562 - acc: 0.9028 - val_loss: 0.2197 - val_acc: 0.9123
Epoch 48/50
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2580 - acc: 0.9002 - val_loss: 0.2202 - val_acc: 0.9045

Epoch 00048: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 49/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2535 - acc: 0.9008 - val_loss: 0.2171 - val_acc: 0.9071
Epoch 50/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2510 - acc: 0.9041 - v

In [ ]:
0.93484